https://www.kaggle.com/krishanudb/tutorial-on-convolutional-nets-in-pytorch  

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [2]:
input_size = [28, 28]
num_epochs = 10
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [5]:
traindata = torchvision.datasets.MNIST(root="./data", train = True, transform=transforms.ToTensor(), download = True)
testdata = torchvision.datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor(), download = True)

100.1%

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


113.5%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


100.4%

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


180.4%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
print(traindata.train_data.size())
print(testdata.test_data.size())

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [7]:
trainloader = torch.utils.data.DataLoader(dataset=traindata, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset=testdata, batch_size=batch_size, shuffle=False)

In [8]:
class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Sequential(nn.Conv2d(1, 6, kernel_size=5, padding = 2),
                                   nn.BatchNorm2d(6),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.conv2 = nn.Sequential(nn.Conv2d(6, 16, kernel_size=3, padding = 2),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2, stride=2))
        self.conv3 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=3, padding = 1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2, stride = 2))
        self.output = nn.Linear(4 * 4 * 32, num_classes)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = out.reshape(-1, 4 * 4 * 32)
        out = self.output(out)
        return out

model = ConvNet(10)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 200 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, len(trainloader), loss.item()))

Epoch [1/10], Step [200/600], Loss: 0.1247
Epoch [1/10], Step [400/600], Loss: 0.0519
Epoch [1/10], Step [600/600], Loss: 0.0516
Epoch [2/10], Step [200/600], Loss: 0.0370
Epoch [2/10], Step [400/600], Loss: 0.0358
Epoch [2/10], Step [600/600], Loss: 0.0806
Epoch [3/10], Step [200/600], Loss: 0.0222
Epoch [3/10], Step [400/600], Loss: 0.0565
Epoch [3/10], Step [600/600], Loss: 0.0372
Epoch [4/10], Step [200/600], Loss: 0.0691
Epoch [4/10], Step [400/600], Loss: 0.0094
Epoch [4/10], Step [600/600], Loss: 0.0603
Epoch [5/10], Step [200/600], Loss: 0.0295
Epoch [5/10], Step [400/600], Loss: 0.0826
Epoch [5/10], Step [600/600], Loss: 0.1098
Epoch [6/10], Step [200/600], Loss: 0.0040
Epoch [6/10], Step [400/600], Loss: 0.0053
Epoch [6/10], Step [600/600], Loss: 0.0138
Epoch [7/10], Step [200/600], Loss: 0.0068
Epoch [7/10], Step [400/600], Loss: 0.0163
Epoch [7/10], Step [600/600], Loss: 0.0261
Epoch [8/10], Step [200/600], Loss: 0.0065
Epoch [8/10], Step [400/600], Loss: 0.0282
Epoch [8/10

In [14]:
with torch.no_grad():
    correct = 0
    total = 0
    for i, (image, labels) in enumerate(testloader):
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
print("Prediction accuracy of the above model is: {}".format(float(correct) / float(total)))

Prediction accuracy of the above model is: 0.9899
